<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [7]</a>'.</span>

In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from transformers import pipeline
import torch
import numpy as np
import os 
from datetime import datetime


2025-10-27 19:49:06.168691: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-27 19:49:06.247988: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2025-10-27 19:49:10.959670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
RANDOM_STATE = 42
path_DATA = "../../../data"
csv_path = f"{path_DATA}/spotify_dataset_sin_duplicados_4.csv"

# To compare with  tfidf
zero_shot_path = f"{path_DATA}/zero_outputs"
os.makedirs(zero_shot_path, exist_ok=True)


TESTING = False

if TESTING:
    NROWS = 50
else:
    NROWS = None
    


In [3]:
def get_array(path):
    with open(path, "r") as f:
        array_ = json.load(f)  
    return array_

def get_song_and_target(csv_path, faltantes_path = "faltantes_according_token.json", sample_size=None):
    df = pd.read_csv(csv_path, nrows=sample_size)
    indices_to_remove = get_array(faltantes_path)
    if indices_to_remove[-1]>df.shape[0]:
        print("You are executing in testing way")
    else:
        df = df.drop(indices_to_remove).reset_index(drop=True)

    df['original_index'] = df.index # Indices correctos despues de la eliminación
    X = df['text']
    df['Explicit_binary'] = (df['Explicit'].str.lower() == 'yes').astype(int)
    y = df['Explicit_binary']
    return X, y, df['original_index']

In [4]:
X,y, index = get_song_and_target(csv_path,"faltantes_according_token.json", sample_size=NROWS)

print(X.shape)
print(y.shape)
print(index.shape)


(108125,)
(108125,)
(108125,)


# meta-llama/Llama-3.1-8B-Instruct
https://huggingface.co/meta-llama/Llama-3.1-8B-Instruct

In [5]:
import os
import transformers
import torch


model_id = "meta-llama/Meta-Llama-3.1-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a expert detecting if a song is explicit or not based on its lyrics. Answer with 'Explicit' or 'Non-Explicit'."},
    {"role": "user", "content": "song lyric: I want to dream in you heart"},
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
)
print(outputs[0]["generated_text"][-1])


`torch_dtype` is deprecated! Use `dtype` instead!


`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Device set to use cuda:0


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


{'role': 'assistant', 'content': 'Non-Explicit'}


In [6]:
df_split_indices = pd.read_csv(f"{zero_shot_path}/split_indices_train.csv")
df_split_indices_test = pd.read_csv(f"{zero_shot_path}/split_indices_test.csv")

idx_train = df_split_indices["train_index"].values
idx_test = df_split_indices_test["test_index"].values

X_train = X.iloc[idx_train].reset_index(drop=True)
X_test = X.iloc[idx_test].reset_index(drop=True)

y_train = y.iloc[idx_train].reset_index(drop=True)
y_test = y.iloc[idx_test].reset_index(drop=True)


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
test_scores = []
for lyric in X_test:
    messages = [
        {"role": "system", "content": "You are an expert detecting if a song is explicit or not based on its lyrics. Answer with 'Explicit' or 'Non-Explicit'."},
        {"role": "user", "content": f"song lyric: {lyric}"},
    ]
    output = pipeline(messages, max_new_tokens=64)
    print()
    # answer = output[0]["generated_text"].strip().split()[-1]
    answer = output[0]["generated_text"][-1]['content']
    score_explicit = 1 if answer.lower() == "explicit" else 0
    test_scores.append(score_explicit)
    break 

test_scores = np.array(test_scores)

# Métricas
f1_test = f1_score(y_test, test_scores)
accuracy_test = accuracy_score(y_test, test_scores)
precision_test = precision_score(y_test, test_scores)
recall_test = recall_score(y_test, test_scores)

df_results = pd.DataFrame({
    "original_index": idx_test,
    "explicit_score": test_scores
})
df_results.to_csv(f"{zero_shot_path}/llama_8B_results.csv", index=False)


print(f"Accuracy en test: {accuracy_test:.8f}")
print(f"Precision en test: {precision_test:.8f}")
print(f"Recall en test: {recall_test:.8f}")
print(f"F1 en test: {f1_test:.8f}")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


ValueError: Found input variables with inconsistent numbers of samples: [21625, 1]